In [3]:
import sys
sys.path

['/Users/larryshamalama/Dropbox/My Mac (Larrys-MBP)/Documents/GitHub/bayesian-programming/PyMC3',
 '/Users/larryshamalama/opt/anaconda3/lib/python38.zip',
 '/Users/larryshamalama/opt/anaconda3/lib/python3.8',
 '/Users/larryshamalama/opt/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/Users/larryshamalama/opt/anaconda3/lib/python3.8/site-packages',
 '/Users/larryshamalama/opt/anaconda3/lib/python3.8/site-packages/aeosa',
 '/Users/larryshamalama/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions',
 '/Users/larryshamalama/.ipython']

In [1]:
import cython
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import pandas as pd

from matplotlib.image import imread

from theano import scan
import theano.tensor as tt

from pymc3.distributions import continuous
from pymc3.distributions import distribution

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-darkgrid')

ModuleNotFoundError: No module named 'pymc3'

In [ ]:
intercept_0 = 4
beta_0 = [2, 3] # the _0 represents the true parameter, not to be confused with the intercept
sigma_0 = 1.5
n = 100

x1 = np.random.normal(loc=3, scale=2, size=[n, 1])
x2 = np.random.binomial(n=1, p=0.5, size=[n, 1])
assert sum(x2) > 0 and sum(x2) < n

x = np.hstack((x1, x2))
y = intercept_0 + x@np.array(beta_0) + np.random.normal(loc=0, scale=sigma_0, size=[n,])

In [ ]:
with pm.Model() as model:
    intercept = pm.Normal("intercept", mu=0, sigma=5)
    
    mean = intercept
    beta = {}
    
    for i in range(x.shape[1]):
        beta[i+1] = pm.Normal(f"beta{i+1}", mu=0, sigma=5)
        mean += beta[i+1]*x[:, i]

    sigma = pm.InverseGamma("sigma", alpha=2, beta=2)
    
    yi = pm.Normal("y", mu=mean, sigma=sigma, observed=y)
    
    trace = pm.sample(draws=2000, chains=3, tune=2000)
    posterior_pred = pm.sample_posterior_predictive(trace)

In [ ]:
trace.get_values("beta1", combine=False)

In [ ]:
_ = pm.traceplot(trace, var_names=["intercept"] + [f"beta{k}" for k in beta.keys()] + ["sigma"])
plt.show()